In [1]:
import numpy as np
import pandas as pd 

import tensorflow as tf
from tensorflow.keras import layers
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import to_categorical, plot_model

import matplotlib.pyplot as plt

from sklearn.neural_network import MLPClassifier

from sklearn.metrics import accuracy_score

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV

from multiprocessing import cpu_count

In [2]:
iris_data = pd.read_csv("data\iris.data", header=None)

In [3]:
iris_data.columns=['sepal length', 'sepal width', 'petal length', 'petal width', 'iris species']

In [4]:
#iris_data

In [5]:
# 80% train, 20% test data
#total_length=len(iris_data)
#train_length=int(0.8*total_length)
#test_length=int(0.2*total_length)

#iris_data_shuffled = iris_data.sample(frac=1)

#train_iris_data = iris_data_shuffled[:train_length]
#test_iris_data = iris_data_shuffled[train_length:]

In [6]:
#print(len(train_iris_data))
#print(len(test_iris_data))

In [7]:
X = iris_data.iloc[:,:4].values
y = iris_data.iloc[:,4:].values

In [8]:
encoder =  LabelEncoder()
y1 = encoder.fit_transform(y)
Y = pd.get_dummies(y1).values

C:\Users\bradl\anaconda3\envs\study\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [9]:
#Y

In [10]:
# 80% train / 20% test
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=0)

In [11]:
# sklearn
clf = MLPClassifier(solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(15,), random_state=1)

In [12]:
clf.fit(X_train, y_train)

C:\Users\bradl\anaconda3\envs\study\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


MLPClassifier(alpha=1e-05, hidden_layer_sizes=(15,), random_state=1,
              solver='lbfgs')

In [13]:
y_pred0 = clf.predict(X_test)

In [14]:
accuracy_score(y_test, y_pred0)

1.0

In [15]:
actual = np.argmax(y_test,axis=1)
predicted = np.argmax(y_pred0,axis=1)
print(f"Actual   : {actual}")
print(f"Predicted: {predicted}")

Actual   : [2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0]
Predicted: [2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0]


In [16]:
# sklearn CV
param_grid = {
    'alpha': [1e-05, 0.001, 0.1, 0.35, 0.5, 0.8, 1],
    'hidden_layer_sizes': [(5,), (10,), (15,), (20,), (4, 3), (5, 3), (5, 4)],
}

cvnn_model = GridSearchCV(estimator=MLPClassifier(solver='lbfgs'),
                        param_grid=param_grid,
                        scoring='accuracy', cv=5,
                        n_jobs=cpu_count(), refit=True)

In [17]:
cvnn_model.fit(X_train, y_train)

C:\Users\bradl\anaconda3\envs\study\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:559: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
  self.n_iter_ = _check_optimize_result("lbfgs", opt_res, self.max_iter)


GridSearchCV(cv=5, estimator=MLPClassifier(solver='lbfgs'), n_jobs=12,
             param_grid={'alpha': [1e-05, 0.001, 0.1, 0.35, 0.5, 0.8, 1],
                         'hidden_layer_sizes': [(5,), (10,), (15,), (20,),
                                                (4, 3), (5, 3), (5, 4)]},
             scoring='accuracy')

In [18]:
print(cvnn_model.best_params_)

{'alpha': 0.1, 'hidden_layer_sizes': (5, 4)}


In [19]:
cvnn_model.score(X_test, y_test)

1.0

In [20]:
# Tensorflow
model = tf.keras.Sequential([
    tf.keras.layers.Dense(40, activation='relu'), # 40 node, hidden layer 1
    #tf.keras.layers.Dense(20, activation='relu'), # 20 node, hidden layer 2
    tf.keras.layers.Dense(3, activation='softmax') # softmax for output, 3 classes -> 3 nodes
  ])
model

In [21]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [22]:
model.fit(X_train, y_train, batch_size=50, epochs=100)

Epoch 1/100
3/3 [==============================] - 1s 3ms/step - loss: 1.5217 - accuracy: 0.1500
Epoch 2/100
3/3 [==============================] - 0s 4ms/step - loss: 1.3584 - accuracy: 0.5917
Epoch 3/100
3/3 [==============================] - 0s 3ms/step - loss: 1.2495 - accuracy: 0.5917
Epoch 4/100
3/3 [==============================] - 0s 3ms/step - loss: 1.1591 - accuracy: 0.6667
Epoch 5/100
3/3 [==============================] - 0s 3ms/step - loss: 1.0888 - accuracy: 0.6917
Epoch 6/100
3/3 [==============================] - 0s 4ms/step - loss: 1.0288 - accuracy: 0.6917
Epoch 7/100
3/3 [==============================] - 0s 3ms/step - loss: 0.9813 - accuracy: 0.6917
Epoch 8/100
3/3 [==============================] - 0s 3ms/step - loss: 0.9392 - accuracy: 0.6917
Epoch 9/100
3/3 [==============================] - 0s 4ms/step - loss: 0.9069 - accuracy: 0.6917
Epoch 10/100
3/3 [==============================] - 0s 4ms/step - loss: 0.8755 - accuracy: 0.6917
Epoch 11/100
3/3 [===========

3/3 [==============================] - 0s 4ms/step - loss: 0.3046 - accuracy: 0.9500
Epoch 85/100
3/3 [==============================] - 0s 3ms/step - loss: 0.3016 - accuracy: 0.9500
Epoch 86/100
3/3 [==============================] - 0s 3ms/step - loss: 0.3012 - accuracy: 0.9417
Epoch 87/100
3/3 [==============================] - 0s 3ms/step - loss: 0.2970 - accuracy: 0.9417
Epoch 88/100
3/3 [==============================] - 0s 3ms/step - loss: 0.2934 - accuracy: 0.9583
Epoch 89/100
3/3 [==============================] - 0s 3ms/step - loss: 0.2912 - accuracy: 0.9500
Epoch 90/100
3/3 [==============================] - 0s 3ms/step - loss: 0.2871 - accuracy: 0.9500
Epoch 91/100
3/3 [==============================] - 0s 4ms/step - loss: 0.2845 - accuracy: 0.9500
Epoch 92/100
3/3 [==============================] - 0s 3ms/step - loss: 0.2813 - accuracy: 0.9500
Epoch 93/100
3/3 [==============================] - 0s 4ms/step - loss: 0.2880 - accuracy: 0.9750
Epoch 94/100
3/3 [===============

In [23]:
loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
print('Test loss:', loss)
print('Test accuracy:', accuracy)

Test loss: 0.25398534536361694
Test accuracy: 1.0


In [24]:
y_pred = model.predict(X_test)
y_pred

1/1 [==============================] - 0s 55ms/step


array([[2.3509575e-04, 1.5604003e-01, 8.4372485e-01],
       [2.4803896e-02, 7.0573020e-01, 2.6946583e-01],
       [9.9196106e-01, 7.8710970e-03, 1.6784285e-04],
       [1.8385555e-04, 2.1585783e-01, 7.8395832e-01],
       [9.7719884e-01, 2.2000588e-02, 8.0050586e-04],
       [6.9042508e-05, 1.0127749e-01, 8.9865345e-01],
       [9.8386842e-01, 1.5610379e-02, 5.2119553e-04],
       [1.0251940e-02, 6.8943638e-01, 3.0031171e-01],
       [7.7106608e-03, 6.5843058e-01, 3.3385876e-01],
       [2.8431900e-02, 7.5816017e-01, 2.1340786e-01],
       [5.3025247e-04, 2.5335893e-01, 7.4611080e-01],
       [1.3944788e-02, 7.0618629e-01, 2.7986890e-01],
       [8.1571955e-03, 6.1362416e-01, 3.7821859e-01],
       [8.0976952e-03, 6.3663036e-01, 3.5527197e-01],
       [6.5098982e-03, 5.7602412e-01, 4.1746601e-01],
       [9.7244477e-01, 2.6405016e-02, 1.1501975e-03],
       [7.8636874e-03, 5.9093720e-01, 4.0119916e-01],
       [8.0937808e-03, 5.5788559e-01, 4.3402061e-01],
       [9.6566254e-01, 3.277

In [25]:
actual = np.argmax(y_test,axis=1)
predicted = np.argmax(y_pred,axis=1)
print(f"Actual   : {actual}")
print(f"Predicted: {predicted}")

Actual   : [2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0]
Predicted: [2 1 0 2 0 2 0 1 1 1 2 1 1 1 1 0 1 1 0 0 2 1 0 0 2 0 0 1 1 0]
